# HOW TO CONNECT TO ORACLE IN JUPYTER

In [ ]:
# dependencies
import os
import time
from datetime import datetime, timedelta
from math import ceil
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# panda's stupid setcopy warning
pd.options.mode.chained_assignment = None

In [ ]:
# oracle
import cx_Oracle
#import psycopg2
#from sqlalchemy import create_engine

In [ ]:
# # pyspark things for Minerva
# from pyspark.sql import HiveContext, SQLContext
# from pyspark.sql.functions import col, count, when

# CONNECTIONS

In [ ]:
# CX_ORACLE
oracle_ezcon = "dukedmts07-scan:1521/sbeaaduke2.world"

# EDW
oracle_URI = "SBEAA.WORLD"
oracle_UID = "TECH_METRICS_READ"
oracle_password = "g8terl8"  

In [ ]:
# MINERVA SPARK CONTEXT
def minerva(sc)
    sc.stop()

    # increase memory
    conf = (SparkConf()
        .set("spark.driver.maxResultSize", "5g"))

    # create contexts
    sc = SparkContext(conf=conf)
    hc = SQLContext(sc)

    return sc, hc

In [ ]:
sc, hc = minerva(sc)
print(sc)
print(hc)

In [ ]:
def ezconnect():
    #setup connection variables for AA
    username = oracle_UID
    password = oracle_password
    URI = oracle_URI
    #connect to the database
    client = cx_Oracle.connect(aa_UID, aa_password, oracle_ezcon)
    cursor = client.cursor()
    return client, cursor

In [ ]:
# test ezconnect()
client, cursor = ezconnect()
print(client)
print(cursor)

## EXAMPLES

In [ ]:
# oracle example

test_sql = """
select *
from siw.dim_site@sbedw.world
where ROWNUM=1
"""

print(test_sql)

In [ ]:
cursor.execute(test_sql)
df_test = pd.read_sql(test_sql, client)
df_test.head()

In [ ]:
# Longer oracle example

date = '9-SEP-19'
interval = 5

sql = """
SELECT
    cust.customer_key,  
    cust.mac_address,
    calls.PARENT_UCID,
    outage.TRAP_TS,
    calls.TOTAL_HANDLED_TIME,
    calls.MINUTE_KEY AS CALL_TIME_KEY,
    trucks.minute_ts as TRUCK_TIME_KEY,
    trucks.trucks_rolled
FROM npm.eh_modem_trans_fact@sbedw.world cust
LEFT JOIN (
    SELECT
        ucef.time_key,
        ucef.minute_key,
        UCEF.CUSTOMER_KEY,
        UCEF.PARENT_UCID,
        UCEF.TOTAL_HANDLED_TIME
    FROM CALL.UNIVERSAL_CALLER_EXP_FACT@SBEDW.WORLD UCEF
    INNER JOIN CALL.FUNCTION_CODE_GRP_LKP@SBEDW.WORLD FCGL ON FCGL.CALL_DICTIONARY_KEY = UCEF.CALL_DICT_FUNCTION_KEY
    WHERE (ucef.INOUT_IND = 1 OR ucef.COX_IDENTITY_SITE_KEY = 31)
    AND ucef.customer_key <> -2
    AND fcgl.BUSINESS_APP LIKE 'Technical Support%'
    AND ucef.time_key >= to_date('{0}') and ucef.time_key < to_date('{0}') + interval '{1}' day
    ) calls ON calls.customer_key = cust.customer_key
left join (
    select
        time_key,
        minute_ts,
        customer_key,
        sum(trouble_call_roll_cnt) as trucks_rolled
    from npm.tech_supp_trans_truck_dtl_fact@sbedw.world
    where PRODUCT_CATEGORY_NM = 'DATA'
    and time_key >= to_date('{0}') and time_key < to_date('{0}') + interval '{1}' day
    group by time_key, minute_ts, customer_key
    ) trucks ON trucks.time_key = cust.time_key
            AND trucks.customer_key = cust.customer_key
INNER JOIN  adv_analytics.JB_PCE_MAC_OUTAGE_CLEARS OUTAGE
    ON OUTAGE.MACADDRESS = cust.mac_address 
WHERE cust.mac_address in (
    SELECT 
        DISTINCT MACADDRESS
    FROM adv_analytics.JB_PCE_MAC_OUTAGE_CLEARS) 
AND cust.time_key >= to_date('{0}') 
AND cust.time_key < to_date('{0}') + interval '{1}' day
""".format(date, interval)

print(sql)

In [ ]:
cursor.execute(sql)
df = pd.read_sql(sql, client)
df.head()

In [ ]:
cursor.close()

In [ ]:
# minerva example

hql = """
select *
from techmetrics.cmts
where condition = "OFFLINE"
and dt >= '{0}'
and dt < '{1}'
limit 10
""".format('2019-09-09','2019-09-10')

print(hql)

In [ ]:
hdf = hc.sql(hql).select("*").toPandas()
hdf.head()